# Import Library

In [26]:
import glob
import os
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from PIL import Image
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split
import seaborn as sns

import warnings
warnings.filterwarnings("ignore")

# Import Data and Normalization

In [27]:
imgs = glob.glob("./img/*.png")

width = 125
height = 50

X = []
Y = []

for img in imgs:
    
    filename = os.path.basename(img)
    label = filename.split("_")[0]
    im = np.array(Image.open(img).convert("L").resize((width,height)))
    im = im / 255 # Normalizasyon
    X.append(im)
    Y.append(label)
    
X = np.array(X)
X = X.reshape(X.shape[0], width, height, 1)

In [28]:
def onehot_labels(values):
    label_encoder = LabelEncoder()
    integer_encoded = label_encoder.fit_transform(values)
    
    onehot_encoder = OneHotEncoder(sparse = False)
    integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
    onehot_encoded = onehot_encoder.fit_transform(integer_encoded)
    return onehot_encoded
Y = onehot_labels(Y)

# Split Data

In [29]:
train_X, test_X, train_y, test_y = train_test_split(X, Y, test_size = 0.25, random_state = 2)
print("Train shape: {}".format(train_X.shape))
print("Test shape: {}".format(test_X.shape))

Train shape: (126, 125, 50, 1)
Test shape: (43, 125, 50, 1)


# Create Model

In [30]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3,3), activation= "relu", input_shape = (width, height, 1)))
model.add(Conv2D(64, kernel_size=(3,3), activation= "relu"))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation="relu"))
model.add(Dropout(0.4))
model.add(Dense(3, activation="softmax"))

# Load Model 

In [ ]:
if os.path.exists("./trex_weight.h5"):
    model.load_weights("trex_weight.h5")
    print("Load Model!")

# Model Training 

In [31]:
model.compile(loss= "categorical_crossentropy", optimizer = "Adam", metrics= ["accuracy"])

In [32]:
model.fit(train_X, train_y, epochs = 35, batch_size = 64)

Epoch 1/35
2/2 [==============================] - 1s 394ms/step - loss: 7.2782 - accuracy: 0.5079
Epoch 2/35
2/2 [==============================] - 1s 323ms/step - loss: 6.4936 - accuracy: 0.4048
Epoch 3/35
2/2 [==============================] - 1s 362ms/step - loss: 2.3856 - accuracy: 0.5476
Epoch 4/35
2/2 [==============================] - 1s 354ms/step - loss: 1.0847 - accuracy: 0.5079
Epoch 5/35
2/2 [==============================] - 1s 321ms/step - loss: 0.9049 - accuracy: 0.4921
Epoch 6/35
2/2 [==============================] - 1s 356ms/step - loss: 0.7599 - accuracy: 0.6032
Epoch 7/35
2/2 [==============================] - 1s 349ms/step - loss: 0.6640 - accuracy: 0.7857
Epoch 8/35
2/2 [==============================] - 1s 321ms/step - loss: 0.5935 - accuracy: 0.8095
Epoch 9/35
2/2 [==============================] - 1s 338ms/step - loss: 0.5627 - accuracy: 0.7619
Epoch 10/35
2/2 [==============================] - 1s 372ms/step - loss: 0.5198 - accuracy: 0.8175
Epoch 11/35
2/2 [==

In [33]:
score_train = model.evaluate(train_X, train_y)
print("Egitim Dogrulugu: %", score_train[1]*100)

score_test = model.evaluate(test_X, test_y)
print("Test Dogrulugu: %", score_test[1]*100)

4/4 [==============================] - 0s 35ms/step - loss: 0.0399 - accuracy: 1.0000
Egitim Dogrulugu: % 100.0
2/2 [==============================] - 0s 18ms/step - loss: 0.0976 - accuracy: 0.9767
Test Dogrulugu: % 97.67441749572754


# Model Save

In [36]:
open("model.json", "w").write(model.to_json())
model.save_weights("trex_weight.h5")